In [4]:
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
matrix_dir = '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/DMRMatrix/AggLayerDMR/'
matrix_list = list(pathlib.Path(matrix_dir).glob('*tsv.gz'))

In [14]:
records = {}
for path in matrix_list:
    dmr_type, bw_type = path.name.split('DMR_')
    bw_type = bw_type[:-9]
    records[(dmr_type, bw_type)] = path
bw_paths = pd.Series(records)

In [42]:
layers = ['IT-L23', 'IT-L4', 'IT-L5', 'IT-L6']

In [175]:
matrix_records = {}
for layer1 in layers:
    matrix_records[layer1] = {}
    matrix = pd.read_csv(bw_paths[(layer1, layer1)], nrows=100000, 
                             comment='@', sep='\t', header=None).iloc[:, 6:]
    nan_rate = matrix.isna().sum(axis=1) / matrix.shape[1]
    total_na_judge = nan_rate < nan_rate.quantile(0.8)
    matrix_records[layer1][layer1] = matrix[total_na_judge].copy()
    
    for layer2 in layers:
        print(layer1, layer2)
        if layer1 == layer2:
            continue
        matrix = pd.read_csv(bw_paths[(layer1, layer2)], nrows=100000, 
                             comment='@', sep='\t', header=None).iloc[:, 6:]
        matrix_records[layer1][layer2] = matrix[total_na_judge].copy()
    break

IT-L23 IT-L23
IT-L23 IT-L4
IT-L23 IT-L5
IT-L23 IT-L6


In [176]:
# filtered_records = {}
# for layer1 in layers:
#     judges = []
#     filtered_records[layer1] = {}
#     hypo_matrix = matrix_records[layer1][layer1]
#     judges.append((hypo_matrix.iloc[97:-97].mean(axis=1).reindex(hypo_matrix.index) < 0.5).values)
#     print(hypo_matrix.shape)
#     for layer2 in layers:
#         if layer2 == layer1:
#             continue
#         hyper_matrix = matrix_records[layer1][layer2]
#         print(hyper_matrix.shape)
#         judges.append((hyper_matrix.iloc[97:-97].mean(axis=1).reindex(hyper_matrix.index) > 0.5).values)
#     total_judge = np.array(np.all(judges, axis=0))
#     hypo_matrix
#     
#     for layer2 in layers:
#         matrix = matrix_records[layer1][layer2]
#         filtered_records[layer1][layer2] = matrix.loc[total_judge]
#     break

In [ ]:
for i in judges:
    print(is)

In [ ]:
cmap = 'coolwarm'
vmin = 0.5
vmax = 0.8

fig = plt.figure(figsize=(2, 4), dpi=300)
gs = fig.add_gridspec(4, 4)

for i, layer1 in enumerate(['IT-L23']):
    for j, layer2 in enumerate(layers):
        ax = fig.add_subplot(gs[i, j])
        matrix = matrix_records[layer1][layer2]
        matrix = matrix.loc[matrix.iloc[:, 30:-30].mean(axis=1).sort_values().index]
        # matrix.mean(axis=0).plot(ax=ax)
        ax.imshow(matrix.values, aspect='auto',
                  cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set(xticks=[], yticks=[])